# TODO 
1. Add streamlit demo 
2. Ignite version 
3. Huggingface Trainer() version 



# What will you learn 

# References 

# reach me! 



## Required Dependencies

In [2]:
# !pip install -U datasets huggingface-hub --quiet
# !pip install -U accelerate --quiet


^C
ERROR: Operation cancelled by user


Before we dive in, we will seed everything using [`manual_seed`](https://pytorch.org/ignite/utils.html#ignite.utils.manual_seed).

In [3]:
from ignite.utils import manual_seed

manual_seed(42)

## Basic Setup

Next we will follow the tutorial and load up our dataset and tokenizer to prepocess the data.

### Data Preprocessing

In [4]:
from datasets import load_dataset

ds = load_dataset("Abdelkareem/arabic_tweets_classification")

Found cached dataset parquet (/home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
ds["train"][0]

{'Date': '2021-03-02 00:00:00',
 'Time': '06:48:15',
 'Date Time': 'Tue Mar 02 06:48:15 +0000 2021',
 'URL': 'https://twitter.com/AlArabiya/status/1366641482610970626',
 'Tweet Text': 'سي إن إن تستعد إدارة الرئيس بايدن لفرض عقوبات على روسيا على خلفية تسميم المعارض لكرملين أليكسينافالني وسجنه العربية',
 'Cleaned Text': 'سي ان ان تستعد اداره الرئيس بايدن لفرض عقوبات علي روسيا علي خلفيه تسميم المعارض لكرملين اليكسينافالني وسجنه العربيه',
 'User Name': 'AlArabiya',
 'Location': None,
 'Replied Tweet ID ': None,
 'Replied Tweet User ID': None,
 'Replied Tweet User name': None,
 'Coordinates': None,
 'Retweet Count': 4.0,
 'Favorite Count': 20,
 'Favorited': 'False',
 'Label': 'Ham'}

In [6]:
ds["train"].features

{'Date': Value(dtype='string', id=None),
 'Time': Value(dtype='string', id=None),
 'Date Time': Value(dtype='string', id=None),
 'URL': Value(dtype='string', id=None),
 'Tweet Text': Value(dtype='string', id=None),
 'Cleaned Text': Value(dtype='string', id=None),
 'User Name': Value(dtype='string', id=None),
 'Location': Value(dtype='string', id=None),
 'Replied Tweet ID ': Value(dtype='float64', id=None),
 'Replied Tweet User ID': Value(dtype='float64', id=None),
 'Replied Tweet User name': Value(dtype='string', id=None),
 'Coordinates': Value(dtype='float64', id=None),
 'Retweet Count': Value(dtype='float64', id=None),
 'Favorite Count': Value(dtype='int64', id=None),
 'Favorited': Value(dtype='string', id=None),
 'Label': Value(dtype='string', id=None)}

In [7]:
columns_to_rmv = [
    x for x in ds["train"].features if "Cleaned Text" not in x and "Label" not in x
]
columns_to_rmv

['Date',
 'Time',
 'Date Time',
 'URL',
 'Tweet Text',
 'User Name',
 'Location',
 'Replied Tweet ID ',
 'Replied Tweet User ID',
 'Replied Tweet User name',
 'Coordinates',
 'Retweet Count',
 'Favorite Count',
 'Favorited']

In [8]:
ds = ds.remove_columns(columns_to_rmv)
ds

DatasetDict({
    train: Dataset({
        features: ['Cleaned Text', 'Label'],
        num_rows: 13240
    })
})

In [9]:
ds = ds.rename_column("Cleaned Text", "text")
ds = ds.rename_column("Label", "label")

In [10]:
import pandas as pd

ds = ds.filter(lambda example: not pd.isna(example["text"]))

Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-8d975b86f0faa411.arrow


In [11]:
ds = ds.class_encode_column("label")
ds

Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5c3d704984785031.arrow
Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-26046df6bb2808f5.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 13239
    })
})

In [12]:
ds["train"][0]

{'text': 'سي ان ان تستعد اداره الرئيس بايدن لفرض عقوبات علي روسيا علي خلفيه تسميم المعارض لكرملين اليكسينافالني وسجنه العربيه',
 'label': 0}

In [13]:
import pandas as pd

ds = ds.filter(lambda example: not pd.isna(example["text"]))
ds = ds.filter(lambda example: not pd.isna(example["label"]))

Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-fb630286263255ff.arrow
Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-ef1db322f1eb6f30.arrow


In [14]:
from ignite.utils import manual_seed

manual_seed(42)

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

# load your pre_trained model with all its weights
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenize_ds = ds.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-efb3990b1d63476c.arrow


In [17]:
tokenize_ds = tokenize_ds["train"].train_test_split(0.2)

Loading cached split indices for dataset at /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-63722e21edaa4a32.arrow and /home/kareem/.cache/huggingface/datasets/Abdelkareem___parquet/Abdelkareem--arabic_tweets_classification-4af7cd4f6467e781/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5fee789880e2423b.arrow


We move towards the end of the tutorial for PyTorch specific instructions. Here we are extracting a larger subset of our original datasets. We also don't need to provide a seed since we seeded everything at the beginning.

In [18]:
tokenize_ds = tokenize_ds.remove_columns(["text"])
tokenize_ds = tokenize_ds.rename_column("label", "labels")
tokenize_ds.set_format("torch")

In [19]:
# small_train_dataset = tokenize_ds["train"].shuffle().select(range(2000))
# small_eval_dataset = tokenize_ds["test"].shuffle().select(range(2000))
small_train_dataset = tokenize_ds["train"].shuffle()
small_eval_dataset = tokenize_ds["test"].shuffle()

### Dataloaders

In [63]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=2)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=2)

### Model

In [64]:
next(iter(train_dataloader))

{'labels': tensor([0, 0]),
 'input_ids': tensor([[   2, 3651, 1136,  ...,    0,    0,    0],
         [   2, 2158, 4492,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [65]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoModel,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, from_tf=False
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-bas

### Optimizer

In [66]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/home/kareem/mambaforge/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### LR Scheduler

We will use the built-in Ignite alternative of `linear` scheduler which is [`PiecewiseLinear`](https://pytorch.org/ignite/generated/ignite.handlers.param_scheduler.PiecewiseLinear.html#piecewiselinear). We will also increase the number of epochs.

In [67]:
from ignite.contrib.handlers import PiecewiseLinear

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)

milestones_values = [
    (0, 5e-5),
    (num_training_steps, 0.0),
]
lr_scheduler = PiecewiseLinear(
    optimizer, param_name="lr", milestones_values=milestones_values
)

### Set Device

In [68]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import torch                                                                                 │
│   2                                                                                              │
│   3 device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")          │
│ ❱ 4 model.to(device)                                                                             │
│   5                                                                                              │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/transformers/modeling_utils.py:1886 in to   │
│                                                                                                  │
│   1883 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1884 │   │   │   )                                                                             │
│   1885 │   │   else:                                                                             │
│ ❱ 1886 │   │   │   return super().to(*args, **kwargs)                                            │
│   1887 │                                                                                         │
│   1888 │   def half(self, *args):                                                                │
│   1889 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/nn/modules/module.py:989 in to        │
│                                                                                                  │
│    986 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│    987 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│    988 │   │                                                                                     │
│ ❱  989 │   │   return self._apply(convert)                                                       │
│    990 │                                                                                         │
│    991 │   def register_backward_hook(                                                           │
│    992 │   │   self, hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]           │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/nn/modules/module.py:641 in _apply    │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/

## Create Trainer

Ignite's [`Engine`](https://pytorch-ignite.ai/concepts/01-engine/) allows users to define a `process_function` to process a given batch of data. This function is applied to all the batches of the dataset. This is a general class that can be applied to train and validate models. A `process_function` has two parameters `engine` and `batch`.

The code for processing a batch of training data in the tutorial is as follows:

```python
for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
```

Therefore we will define a `process_function` (called `train_step` below) to do the above tasks:

* Set `model` in train mode.
* Move items of the `batch` to `device`.
* Perform forward pass and generate `output`.
* Extract loss.
* Perform backward pass using loss to calculate gradients for the model parameters.
* Optimize model parameters using gradients and optimizer.

Finally, we choose to return the `loss` so we can utilize it for futher processing.

You will also notice that we do not update the `lr_scheduler` and `progress_bar` in `train_step`. This is because Ignite automatically takes care of it as we will see later in this tutorial.

In [69]:
def train_step(engine, batch):
    model.train()
    #     print(batch)
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    return loss

And then we create a model `trainer` by attaching the `train_step` to the training engine. Later, we will use `trainer` for looping over the training dataset for `num_epochs`.

In [70]:
from ignite.engine import Engine

trainer = Engine(train_step)

The `lr_scheduler` we defined perviously was a handler.

[Handlers](https://pytorch-ignite.ai/concepts/02-events-and-handlers/#handlers) can be any type of function (lambda functions, class methods, etc). On top of that, Ignite provides several built-in handlers to reduce redundant code. We attach these handlers to engine which is triggered at a specific [event](https://pytorch-ignite.ai/concepts/02-events-and-handlers/). These events can be anything like the start of an iteration or the end of an epoch. [Here](https://pytorch.org/ignite/generated/ignite.engine.events.Events.html#events) is a complete list of built-in events.

Therefore, we will attach the `lr_scheduler` (handler) to the `trainer` (`engine`) via [`add_event_handler()`](https://pytorch.org/ignite/generated/ignite.engine.engine.Engine.html#ignite.engine.engine.Engine.add_event_handler) so it can be triggered at `Events.ITERATION_STARTED` (start of an iteration) automatically.

In [71]:
from ignite.engine import Events

trainer.add_event_handler(Events.ITERATION_STARTED, lr_scheduler)

This is the reason we did not include `lr_scheduler.step()` in `train_step()`.

## Progress Bar

Next we create an instance of Ignite's [`ProgessBar()`](https://pytorch.org/ignite/generated/ignite.contrib.handlers.tqdm_logger.html#ignite.contrib.handlers.tqdm_logger.ProgressBar) and attach it to the trainer to replace `progress_bar.update(1)`.

In [72]:
from ignite.contrib.handlers import ProgressBar

pbar = ProgressBar()

We can either, simply track the progress:

In [73]:
pbar.attach(trainer)

Or also track the output of `trainer` (or `train_step`):

In [74]:
pbar.attach(trainer, output_transform=lambda x: {"loss": x})

## Create Evaluator

Similar to the training `process_function`, we setup a function to evaluate a single batch of train/validation/test data.

```python
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
```

 Here is what `evaluate_step()` below does:

* Sets model in eval mode.
* Move items of the `batch` to `device`.
* With `torch.no_grad()`, no gradients are calculated for any succeding steps.
* Perform a forward pass on the model to calculate `outputs` from `batch`
* Get the real `predictions` from `logits` (probability of positive and negative classes).

Finally, we return the predictions and the actual labels so that we can compute the metrics.

You will notice that we did not compute the metrics in `evaluate_step()`. This is because Ignite provides built-in [metrics](https://pytorch-ignite.ai/concepts/04-metrics/) which we can later attach to the engine.

**Note:** Ignite suggests attaching metrics to evaluators and not trainers because during the training the model parameters are constantly changing and it is best to evaluate model on a stationary model. This information is important as there is a difference in the functions for training and evaluating. Training returns a single scalar loss. Evaluating returns `y_pred` and `y` as that output is used to calculate metrics per batch for the entire dataset.

All metrics in Ignite require `y_pred` and `y` as outputs of the function attached to the Engine.

In [75]:
def evaluate_step(engine, batch):
    model.eval()

    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    return {"y_pred": predictions, "y": batch["labels"]}

Below we create two engines, a training evaluator and a validation evaluator. `train_evaluator` and `validation_evaluator` use the same function but they serve different purposes as we will see later in this tutorial.

In [76]:
train_evaluator = Engine(evaluate_step)
validation_evaluator = Engine(evaluate_step)

## Attach Metrics

The 🤗 tutorial defines one metric, accuracy, to be used for evaluation:

```python
metric= load_metric("accuracy")
```

We can easily attach Ignite's built-in [`Accuracy()`](https://pytorch.org/ignite/generated/ignite.metrics.Accuracy.html#accuracy) metric to to `train_evaluator` and `validation_evaluator`. We also need to specify the metric name (`accuracy` below). Internally, it will use `y_pred` and `y` to compute the accuracy.

In [77]:
from ignite.metrics import Accuracy

Accuracy().attach(train_evaluator, "accuracy")
Accuracy().attach(validation_evaluator, "accuracy")

## Log Metrics

Now we will define custom handlers (functions) and attach them to various `Events` of the training process.

The functions below both achieve similar tasks. They print the results of the `evaluator` run on a dataset. `log_training_results()` does this on the training evaluator and train dataset, while `log_validation_results()` on the validation evaluator and validation dataset. Another difference is how these functions are attached in the trainer engine.

The first method involves using a decorator, the syntax is simple - `@` `trainer.on(Events.EPOCH_COMPLETED)`, means that the decorated function will be attached to the trainer and called at the end of each epoch.

The second method involves using the add_event_handler method of trainer - `trainer.add_event_handler(Events.EPOCH_COMPLETED, custom_function)`. This achieves the same result as the above.

In [78]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    print(
        f"Training Results - Epoch: {engine.state.epoch}  Avg accuracy: {avg_accuracy:.3f}"
    )


def log_validation_results(engine):
    validation_evaluator.run(eval_dataloader)
    metrics = validation_evaluator.state.metrics
    avg_accuracy = metrics["accuracy"]
    print(
        f"Validation Results - Epoch: {engine.state.epoch}  Avg accuracy: {avg_accuracy:.3f}"
    )


trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)

## Early Stopping

Now we'll setup a [`EarlyStopping`](https://pytorch.org/ignite/generated/ignite.handlers.early_stopping.EarlyStopping.html#earlystopping) handler for the training process. `EarlyStopping` requires a score_function that allows the user to define whatever criteria to stop trainig. In this case, if the loss of the validation set does not decrease in 2 epochs (`patience`), the training process will stop early.  

In [79]:
from ignite.handlers import EarlyStopping


def score_function(engine):
    val_accuracy = engine.state.metrics["accuracy"]
    return val_accuracy


handler = EarlyStopping(patience=2, score_function=score_function, trainer=trainer)
validation_evaluator.add_event_handler(Events.COMPLETED, handler)

## Model Checkpoint

Lastly, we want to save the best model weights. So we will use Ignite's [`ModelCheckpoint`](https://pytorch.org/ignite/generated/ignite.handlers.checkpoint.ModelCheckpoint.html#modelcheckpoint) handler to checkpoint models at the end of each epoch. This will create a `models` directory and save the 2 best models (`n_saved`) with the prefix `bert-base-cased`.

In [80]:
from ignite.handlers import ModelCheckpoint

checkpointer = ModelCheckpoint(
    dirname="models",
    filename_prefix="bert_tweets_classification",
    n_saved=2,
    create_dir=True,
)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {"model": model})

## Begin Training!

Next, we'll run the trainer for 10 epochs and monitor the results. Below we can see that `ProgessBar` prints the loss per iteration, and prints the results of training and validation as we specified in our custom function.

In [86]:
import torch

torch.cuda.empty_cache()

In [87]:
model = model.to(device)
trainer.run(train_dataloader, max_epochs=_epochs)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model = model.to(device)                                                                     │
│   2 trainer.run(train_dataloader, max_epochs=_epochs)                                            │
│   3                                                                                              │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/transformers/modeling_utils.py:1886 in to   │
│                                                                                                  │
│   1883 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1884 │   │   │   )                                                                             │
│   1885 │   │   else:                                                                             │
│ ❱ 1886 │   │   │   return super().to(*args, **kwargs)                                            │
│   1887 │                                                                                         │
│   1888 │   def half(self, *args):                                                                │
│   1889 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/nn/modules/module.py:989 in to        │
│                                                                                                  │
│    986 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│    987 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│    988 │   │                                                                                     │
│ ❱  989 │   │   return self._apply(convert)                                                       │
│    990 │                                                                                         │
│    991 │   def register_backward_hook(                                                           │
│    992 │   │   self, hook: Callable[['Module', _grad_t, _grad_t], Union[None, Tensor]]           │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/nn/modules/module.py:641 in _apply    │
│                                                                                                  │
│    638 │                                                                                         │
│    639 │   def _apply(self, fn):                                                                 │
│    640 │   │   for module in self.children():                                                    │
│ ❱  641 │   │   │   module._apply(fn)                                                             │
│    642 │   │                                                                                     │
│    643 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    644 │   │   │   if torch._has_compatible_shallow_copy_type(tensor, tensor_applied):           │
│                                                                                                  │
│ /home/kareem/mambaforge/lib/python3.10/site-packages/torch/nn/modules/module.py:641 in _apply    │
│                                                                                                  │
│    638 │                                                   

That's it! We have successfully trained and evaluated a Transformer for Text Classification.

In [90]:
# import os 
# print(sorted(os.listdir('models')))
# print(handler.last_checkpoint)
# from ignite.handlers import Checkpoint
# checkpoint_dir =  "/kaggle/working/models/"

checkpoint_fp = checkpoint_dir + "bert_tweets_classification_model_1324.pt"
checkpoint = torch.load(checkpoint_fp, map_location=device) 
# Checkpoint.load_objects(to_load=to_save, checkpoint=checkpoint) 


In [93]:
type(checkpoint)
from transformers import BertTokenizer
model = BertTokenizer.from_pretrained(checkpoint_fp,local_files_only=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1727: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_28/460194186.py:3 in <module>                                                     │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28/460194186.py'                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1825 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   1822 │   │   │   else:                                                                         │
│   1823 │   │   │   │   logger.info(f"loading file {file_path} from cache at {resolved_vocab_fil  │
│   1824 │   │                                                                                     │
│ ❱ 1825 │   │   return cls._from_pretrained(                                                      │
│   1826 │   │   │   resolved_vocab_files,                                                         │
│   1827 │   │   │   pretrained_model_name_or_path,                                                │
│   1828 │   │   │   init_configuration,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1988 in          │
│ _from_pretrained                                                                                 │
│                                                                                                  │
│   1985 │   │                                                                                     │
│   1986 │   │   # Instantiate tokenizer.                                                          │
│   1987 │   │   try:                                                                              │
│ ❱ 1988 │   │   │   tokenizer = cls(*init_inputs, **init_kwargs)                                  │
│   1989 │   │   except OSError:                                                                   │
│   1990 │   │   │   raise OSError(                                                                │
│   1991 │   │   │   │   "Unable to load vocabulary from file. "                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/bert/tokenization_bert.py:218 in     │
│ __init__                                                                                         │
│                                                                                                  │
│   215 │   │   │   │   f"Can't find a vocabulary file at path '{vocab_file}'. To load the vocab   │
│   216 │   │   │   │   " model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_   │
│   217 │   │   │   )                                                                              │
│ ❱ 218 │   │   self.vocab = load_vocab(vocab_file)                                                │
│   219 │   │   self.ids_to_tokens = collections.OrderedDict([(ids, tok) for tok, ids in self.vo   │
│   220 │   │   self.do_basic_tokenize = do_basic_tokenize                                         │
│   221 │   │   if do_basic_tokenize:                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/bert/tokenization_bert.py:121 in     │
│ load_vocab                                                 

In [ ]:
model.push_to_hub("arabert_tweets_classification")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_28/1329976981.py:7 in <module>                                                    │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28/1329976981.py'                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Checkpoint.load_objects() missing 2 required positional arguments: 'to_load' and 'checkpoint'